In [ ]:
#CONSTANTS

TMin=5
TMax=15
w=2*pi/24
k=0.1
stepSize=.05


#UTILITATS

def integrarPunts(l,stepSize):
    area=0
    for i in range(len(l)-1):
        area+=(l[i][1]+l[i+1][1])/2 #àrea trapezi
    return area*stepSize

def Te(t): #temperatura ambient
    return (TMin+TMax)/2+(TMax-TMin)*sin(w*t+pi)/2

def free(temperaturaIni,tIni,k): #solució analítica evolució lliure
    return desolve(diff(T(t),t)==-k*(T(t)-Te(t)),[T(t),t],ics=[tIni,temperaturaIni]).expand()

#MODELS 


def calorTotDia(T,TSup): #Calor termòstat, s'apaga si la temperatura supera TSup
    return (TSup-T)*(unit_step(T)-unit_step(T-TSup))


def termostatTotDia(r,horaFinal,horaInicial,Tinicial,restriccio,TSup): #Per ajustar els paràmetres r als diversos models
    #diu si amb una r donada s'arriba a la temperatura desitjada, i en cas afirmatiu, quant es gasta
    eq2=diff(T(t),t)==(r*calorTotDia(T(t),TSup)-k*(T(t)-Te(t)))
    sol2=desolve_rk4(eq2, T, step=stepSize, ics=[horaInicial,Tinicial],end_points=horaFinal)
    if sol2[-1][1]<restriccio:
        return 0,0,0
    
    q2=[(i[0],r*calorTotDia(i[1])) for i in sol2]
    energiaGastada=integrarPunts(q2,stepSize).n() #energia gastada
    return sol2,energiaGastada,q2





def calorTram(Tini,tf,tini,k): #evolució lliure començant a tini amb temperatura Tini fins a tf amb k paràmetre
    sol=free(Tini,tini,k) # per model 3
    alpha=1-0.1/k
    dsol=diff(sol,t)
    temp=[(i,sol(i)) for i in srange(tini,tf,stepSize)]
    calor=[(i,alpha*dsol(i)) for i in srange(tini,tf,stepSize)]
    return temp,calor




def model4(tSD,tID,tSN,tIN):
    horaEntrada=17

    horaEntrada=17
    horaDormir=23
    horaDespertar=7+24
    horaSortir=9+24
    punts=[]
    tA=horaEntrada
    tSupDia=tSD
    tInfDia=tID
    tSupNit=tSN
    tInfNit=tIN
    despesa=0


    while(tA<horaDormir):
        evolucionLibre=desolve(diff(T(t),t)==-k*(T(t)-Te(t)),[T(t),t],ics=[tA,tSupDia]).expand()
        try:#miro cuanto tarda en pasar de tSupDia a tInfDia
            tiempoTarda=find_root(evolucionLibre-tInfDia,tA,horaDormir)
            punts+=list((map(lambda x:(x,evolucionLibre(x).n()),srange(tA,tiempoTarda,stepSize))))

            tA=tiempoTarda
            despesa=tSupDia-tInfDia+despesa

        except:
            #Ja no s'arriba a tInfDia mes
            break

    tB=find_root(evolucionLibre-tInfNit,tA,horaDespertar)
    punts+=list((map(lambda x:(x,evolucionLibre(x).n()),srange(tA,tB,stepSize))))
    tA=tB
    while(tA<horaDespertar):
        evolucionLibre=desolve(diff(T(t),t)==-k*(T(t)-Te(t)),[T(t),t],ics=[tA,tSupNit]).expand()
        try:
            tiempoTarda=find_root(evolucionLibre-tInfNit,tA,horaDespertar)#miro cuanto tarda en pasar de 

            punts+=list((map(lambda x:(x,evolucionLibre(x).n()),srange(tA,tiempoTarda,stepSize))))
            tA=tiempoTarda
            despesa=tSupNit-tInfNit+despesa
        except:
            #Ja no s'arriba a tInfNit es
            break    



    punts+=list((map(lambda x:(x,evolucionLibre(x).n()),srange(tA,horaDespertar,stepSize))))
    despesa+=tSupDia-evolucionLibre(horaDespertar)
    tA=horaDespertar
    while(tA<horaSortir):
        evolucionLibre=desolve(diff(T(t),t)==-k*(T(t)-Te(t)),[T(t),t],ics=[tA,tSupDia]).expand()
        try:
            tiempoTarda=find_root(evolucionLibre-tInfDia,tA,horaSortir)#miro cuanto tarda en pasar de 

            punts+=list((map(lambda x:(x,evolucionLibre(x).n()),srange(tA,tiempoTarda,stepSize))))
            tA=tiempoTarda
            despesa=tSupDia-tInfDia+despesa
        except:
            punts+=list((map(lambda x:(x,evolucionLibre(x).n()),srange(tA,horaSortir+stepSize,stepSize))))
            break
   #ho poso aqui per periodicitat ,el primer dia sera diferent pero a partir del segon sera aixi
    despesa+=tSupDia-evolucionLibre(horaEntrada+24) 
    return despesa,punts